# Machine Learning Practical: Coursework 2
## Experiment 1/3

**Due date: 16:00 Thursday 24th November 2016**

# Georgios Pligoropoulos - s1687568

In [1]:
from numba import autojit

In [2]:
def checkVarIsDefined(varName):
    try:
        exec(varName)
    except NameError:
        return False
    else:
        return True

In [3]:
import sys
mlpdir = '/home/student/Dropbox/msc_Artificial_Intelligence/mlp_Machine_Learning_Practical/mlpractical'
sys.path.append(mlpdir)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import logging

#%matplotlib notebook
%matplotlib inline

In [5]:
from mlp.data_providers import MNISTDataProvider

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer

#from mlp.errors import CrossEntropySoftmaxError

#from mlp.models import MultipleLayerModel

from mlp.initialisers import ConstantInit, GlorotUniformInit

#from mlp.optimisers import Optimiser

from mlp.learning_rules import GradientDescentLearningRule, MomentumLearningRule, \
    AdaGradLearningRule, RmsPropLearningRule
    
from mlp.schedulers import MomentumCoefficientScheduler, ConstantLearningRateScheduler,\
    ReciprocalLearningRateScheduler, ExponentialLearningRateScheduler

In [6]:
class CrossEntropySoftmaxError(object):
    """Multi-class cross entropy error with Softmax applied to outputs."""

    def __call__(self, outputs, targets):
        """Calculates error function given a batch of outputs and targets.

        Args:
            outputs: Array of model outputs of shape (batch_size, output_dim).
            targets: Array of target outputs of shape (batch_size, output_dim).

        Returns:
            Scalar error function value.
        """
        # subtract max inside exponential to improve numerical stability -
        # when we divide through by sum this term cancels
        probs = np.exp(outputs - outputs.max(-1)[:, None])
        probs /= probs.sum(-1)[:, None]
        return -np.mean(np.sum(targets * np.log(probs), axis=1))

    @autojit
    def grad(self, outputs, targets):
        """Calculates gradient of error function with respect to outputs.

        Args:
            outputs: Array of model outputs of shape (batch_size, output_dim).
            targets: Array of target outputs of shape (batch_size, output_dim).

        Returns:
            Gradient of error function with respect to outputs.
        """
        # subtract max inside exponential to improve numerical stability -
        # when we divide through by sum this term cancels
        probs = np.exp(outputs - outputs.max(-1)[:, None])
        probs /= probs.sum(-1)[:, None]
        return (probs - targets) / outputs.shape[0]

    def __repr__(self):
        return 'CrossEntropySoftmaxError'

In [7]:
from mlp.layers import LayerWithParameters, StochasticLayer

class MultipleLayerModel(object):
    """A model consisting of multiple layers applied sequentially."""

    def __init__(self, layers):
        """Create a new multiple layer model instance.

        Args:
            layers: List of the the layer objecst defining the model in the
                order they should be applied from inputs to outputs.
        """
        self.layers = layers

    @property
    def params(self):
        """A list of all of the parameters of the model."""
        params = []
        for layer in self.layers:
            if isinstance(layer, LayerWithParameters):
                params += layer.params
        return params

    @autojit
    def fprop(self, inputs, stochastic=True):
        """Forward propagates a batch of inputs through the model.

        Args:
            inputs: Batch of inputs to the model.
            stochastic: Whether to use stochastic forward propagation
                for stochastic layers (True) or deterministic (False).

        Returns:
            List of the activations at the output of all layers of the model
            plus the inputs (to the first layer) as the first element. The
            last element of the list corresponds to the model outputs.
        """
        activations = [inputs]
        for i, layer in enumerate(self.layers):
            if isinstance(layer, StochasticLayer):
                activations.append(self.layers[i].fprop(
                    activations[i], stochastic=stochastic))
            else:
                activations.append(self.layers[i].fprop(activations[i]))
        return activations

    #@autojit
    #@jit(nopython=True)
    @autojit(nopython=True)
    def grads_wrt_params(self, activations, grads_wrt_outputs):
        """Calculates gradients with respect to the model parameters.

        Args:
            activations: List of all activations from forward pass through
                model using `fprop`.
            grads_wrt_outputs: Gradient with respect to the model outputs of
               the scalar function parameter gradients are being calculated
               for.

        Returns:
            List of gradients of the scalar function with respect to all model
            parameters.
        """
        grads_wrt_params = []
        for i, layer in enumerate(self.layers[::-1]):
            inputs = activations[-i - 2]
            outputs = activations[-i - 1]
            if isinstance(layer, LayerWithParameters):
                # Gradients are appended in reversed order, going backwards
                # through layers, so grads_wrt_params can be reversed at end to
                # give gradients in consistent order with self.params
                grads_wrt_params += layer.grads_wrt_params(inputs, grads_wrt_outputs)[::-1]
            # If not at first layer back-propagate gradients
            if i != len(self.layers) - 1:
                grads_wrt_outputs = layer.bprop(
                    inputs, outputs, grads_wrt_outputs)
        return grads_wrt_params[::-1]

    def params_penalty(self):
        """Calculates the parameter dependent penalty term of the model."""
        params_penalty = 0.
        for layer in self.layers:
            if isinstance(layer, LayerWithParameters):
                params_penalty += layer.params_penalty()
        return params_penalty

    def __repr__(self):
        return (
            'MultiLayerModel(\n    ' +
            '\n    '.join([str(layer) for layer in self.layers]) +
            '\n)'
        )


In [8]:
import time
import logging
from collections import OrderedDict
import numpy as np


logger = logging.getLogger(__name__)


class Optimiser(object):
    """Basic model optimiser."""

    def __init__(self, model, error, learning_rule, train_dataset,
                 valid_dataset=None, data_monitors=None, schedulers=[],
                 use_stochastic_eval=True):
        """Create a new optimiser instance.

        Args:
            model: The model to optimise.
            error: The scalar error function to minimise.
            learning_rule: Gradient based learning rule to use to minimise
                error.
            train_dataset: Data provider for training set data batches.
            valid_dataset: Data provider for validation set data batches.
            data_monitors: Dictionary of functions evaluated on targets and
                model outputs (averaged across both full training and
                validation data sets) to monitor during training in addition
                to the error. Keys should correspond to a string label for
                the statistic being evaluated.
            schedulers: List of learning rule scheduler objects for adjusting
                learning rule hyperparameters over training. Can be empty.
            use_stochastic_eval: Whether to use `stochastic=True` flag in
                `model.fprop` for evaluating model performance during training.
        """
        self.model = model
        self.error = error
        self.learning_rule = learning_rule
        self.learning_rule.initialise(self.model.params)
        self.train_dataset = train_dataset
        self.valid_dataset = valid_dataset
        self.data_monitors = OrderedDict([('error', error)])
        if data_monitors is not None:
            self.data_monitors.update(data_monitors)
        self.schedulers = schedulers
        self.use_stochastic_eval = use_stochastic_eval

    #@autojit
    def do_training_epoch(self):
        """Do a single training epoch.

        This iterates through all batches in training dataset, for each
        calculating the gradient of the estimated error given the batch with
        respect to all the model parameters and then updates the model
        parameters according to the learning rule.
        """
        for inputs_batch, targets_batch in self.train_dataset:
            activations = self.model.fprop(inputs_batch)
            grads_wrt_outputs = self.error.grad(activations[-1], targets_batch)
            grads_wrt_params = self.model.grads_wrt_params(
                activations, grads_wrt_outputs)
            self.learning_rule.update_params(grads_wrt_params)

    def eval_monitors(self, dataset, label):
        """Evaluates the monitors for the given dataset.

        Args:
            dataset: Dataset to perform evaluation with.
            label: Tag to add to end of monitor keys to identify dataset.

        Returns:
            OrderedDict of monitor values evaluated on dataset.
        """
        data_mon_vals = OrderedDict([(key + label, 0.) for key
                                     in self.data_monitors.keys()])
        for inputs_batch, targets_batch in dataset:
            activations = self.model.fprop(
                inputs_batch, stochastic=self.use_stochastic_eval)
            for key, data_monitor in self.data_monitors.items():
                data_mon_vals[key + label] += data_monitor(
                    activations[-1], targets_batch)
        for key, data_monitor in self.data_monitors.items():
            data_mon_vals[key + label] /= dataset.num_batches
        return data_mon_vals

    def get_epoch_stats(self):
        """Computes training statistics for an epoch.

        Returns:
            An OrderedDict with keys corresponding to the statistic labels and
            values corresponding to the value of the statistic.
        """
        epoch_stats = OrderedDict()
        epoch_stats.update(self.eval_monitors(self.train_dataset, '(train)'))
        if self.valid_dataset is not None:
            epoch_stats.update(self.eval_monitors(
                self.valid_dataset, '(valid)'))
        epoch_stats['params_penalty'] = self.model.params_penalty()
        return epoch_stats

    def log_stats(self, epoch, epoch_time, stats):
        """Outputs stats for a training epoch to a logger.

        Args:
            epoch (int): Epoch counter.
            epoch_time: Time taken in seconds for the epoch to complete.
            stats: Monitored stats for the epoch.
        """
        logger.info('Epoch {0}: {1:.2f}s to complete\n  {2}'.format(
            epoch, epoch_time,
            ', '.join(['{0}={1:.2e}'.format(k, v) for (k, v) in stats.items()])
        ))

    #@autojit
    def train(self, num_epochs, stats_interval=5):
        """Trains a model for a set number of epochs.

        Args:
            num_epochs: Number of epochs (complete passes through trainin
                dataset) to train for.
            stats_interval: Training statistics will be recorded and logged
                every `stats_interval` epochs.

        Returns:
            Tuple with first value being an array of training run statistics,
            the second being a dict mapping the labels for the statistics
            recorded to their column index in the array and the final value
            being the total time elapsed in seconds during the training run.
        """
        stats = self.get_epoch_stats()
        logger.info(
            'Epoch 0:\n  ' +
            ', '.join(['{0}={1:.2e}'.format(k, v) for (k, v) in stats.items()])
        )
        run_stats = [stats.values()]
        run_start_time = time.time()
        for epoch in range(1, num_epochs + 1):
            for scheduler in self.schedulers:
                scheduler.update_learning_rule(self.learning_rule, epoch - 1)
            start_time = time.time()
            self.do_training_epoch()
            epoch_time = time.time() - start_time
            if epoch % stats_interval == 0:
                stats = self.get_epoch_stats()
                self.log_stats(epoch, epoch_time, stats)
                run_stats.append(stats.values())
        run_time = time.time() - run_start_time
        return (
            np.array(run_stats),
            {k: i for i, k in enumerate(stats.keys())},
            run_time
        )
        return np.array(run_stats)

In [9]:
def train_model(model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, schedulers=[]):
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, schedulers = schedulers)

    # Run the optimiser for some number of epochs (full passes through the training set)
    # printing statistics every epoch
    stats, keys, runTime = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
    #stats = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
    
#     keys = {'acc(train)': 1,
#      'acc(valid)': 3,
#      'error(train)': 0,
#      'error(valid)': 2,
#      'params_penalty': 4}
    
#     runTime = 0
    
    return stats, keys, runTime

In [10]:
def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, schedulers=[]):
    
    stats, keys, runTime = train_model(model, error, learning_rule, train_data, valid_data,
                                       num_epochs, stats_interval, schedulers)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    
    return stats, keys, fig_1, ax_1, fig_2, ax_2, runTime

In [11]:
def getFinalValues(statistics):
    lastStats = statistics[-1]
    return {
        "Final Training Error": lastStats[0],
        "Final Training Accuracy": lastStats[1],
        "Final Testing Error": lastStats[2],
        "Final Testing Accuracy": lastStats[3],
    }

In [12]:
seed = 16011984
rng = np.random.RandomState(seed)

Note: When in our explanations refer to **accuracy** we mean the final validation accuracy and when we refer to performance we mean how fast we reached the optimal accuracy for the current experiment

# Grand Experiment 1: Exploring the system architecture in numbers of layers and numbers of nodes

We will explore where the systems breaks/fails for too few layers or too few features and where for too many of them

We are going to a simple system with constant learning rate since here we are not optimizing for accuracy neither optimizing for performance. Rather we want to see how the complexity of the model generally affects the  accuracy and performance on a dataset as complex as MNIST. In other words we care for comparison rather than absolute values.

We will use the constant learning rate of 1.1 because it performed relatively well, with good accuracy and performance without making the system too unstable

Note: we are using a high learning rate that produced good enough results within 20 epochs in the previous experiment

In [13]:
weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

In [14]:
# Seed a random number generator

# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = MNISTDataProvider('train', batch_size=50, rng=rng)
valid_data = MNISTDataProvider('valid', batch_size=50, rng=rng)

error = CrossEntropySoftmaxError() #this does not contain any immutable data that's why it can be set only once

In [15]:
stats_interval = 5

## Constant Learning Rate 1.1

## Three Layers and dimensionality of 100 for each hidden layer

In [16]:
num_epochs = 30

input_dim, output_dim, hidden_dim = 784, 10, 100

model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init),
    SigmoidLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
    SigmoidLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

learning_rate = 1.1
learning_rule = GradientDescentLearningRule(learning_rate=learning_rate)

#stats, keys, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
stats, keys, fig_1, ax_1, fig_2, ax_2, runTime = train_model_and_plot_stats(
    model, error, 
    learning_rule,
    train_data, valid_data, num_epochs, stats_interval,
    schedulers = [ConstantLearningRateScheduler(learning_rate)]
)

print "runtime: " + str(runTime)

getFinalValues(stats)

Epoch 0:
  error(train)=2.77e+00, acc(train)=9.94e-02, error(valid)=2.77e+00, acc(valid)=9.90e-02, params_penalty=0.00e+00


TypeError: Caused By:
Traceback (most recent call last):
  File "/home/student/anaconda2/envs/mlp/lib/python2.7/site-packages/numba/compiler.py", line 249, in run
    stage()
  File "/home/student/anaconda2/envs/mlp/lib/python2.7/site-packages/numba/compiler.py", line 459, in stage_nopython_frontend
    legalize_given_types(self.args, self.return_type)
  File "/home/student/anaconda2/envs/mlp/lib/python2.7/site-packages/numba/compiler.py", line 727, in legalize_given_types
    "mode" % (i, a))
TypeError: Arg 0 of pyobject is not legal in nopython mode

Failed at nopython (nopython frontend)
Arg 0 of pyobject is not legal in nopython mode